In [1]:
import numpy as np
import pandas as pd
import tushare as ts
import MySQLdb as mdb

import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

import pandas as pd
from sklearn import cluster,covariance,manifold

from matplotlib.font_manager import FontProperties

# 导入tushare
import tushare as ts

C:\Users\zengke\AppData\Local\Programs\Python\Python39\lib\site-packages\ipykernel\eventloops.py:256: RuntimeWarning: coroutine 'Kernel.do_one_iteration' was never awaited
  self.func()


In [ ]:

#获取沪深300指数的股票名单
hs300_data=ts.get_hs300s()
#hss=hs300_data[“name”]

#获取上交所SSE，深交所SZSE，港交所HKEX正常上市交易的股票名单
pro=ts.pro_api('84a7d42ba53be1e345b15650d19116113087ad5440320b4c6b7b6c67')
exc=["SSE","SZSE"]
stock_data=[]
for ex in exc:
    data=pro.query('stock_basic', exchange=ex,
                    list_status='',
                    fields='ts_code,symbol,name,area,industry,list_date')
    stock_data.append(data)


In [ ]:

#获取沪深300成分股中正常上市交易的名单
#将stock_data中上交所和深交所中的交易数据合并
s_name=pd.concat([stock_data[0][[“name”,“ts_code”]],
stock_data[1][[“name”,“ts_code”]]],ignore_index=True)

#找出沪深300指在上交所和深交所的交易代码
hs300_data=hs300_data.set_index(“name”)
s_name=s_name.set_index(“name”)

sdata=pd.merge(hs300_data,s_name,on=“name”,how=“inner”)

ts_code=sdata[“ts_code”].values

n=len(ts_code)

#提取沪深300指2010年01月01到2018年01月01的交易数据，存入d_price中
d_price=[]
names=[]
symbols=[]
for i in range(62,199):
df = pro.daily(ts_code=ts_code[i],
start_date=‘20181201’,
end_date=‘20190101’)
d_price.append(df)
names.append(sdata[sdata[“ts_code”]==ts_code[i]].index.tolist())
symbols.append(ts_code[i])

names=pd.DataFrame(names)
symbols=pd.DataFrame(symbols)

op=[]
cl=[]
for q in d_price:
    op.append(q[‘open’].values)
    cl.append(q[‘close’].values)

close_prices=np.vstack([i for i in op])
open_prices=np.vstack([j for j in cl])

The daily variations of the quotes are what carry most information
variation = close_prices - open_prices

Learn a graphical structure from the correlations
edge_model = covariance.GraphicalLassoCV(cv=5)

standardize the time series: using correlations rather than covariance
is more efficient for structure recovery
X = variation.copy().T
X /= X.std(axis=0)
edge_model.fit(X)

Cluster using affinity propagation
, labels = cluster.affinity_propagation(edge_model.covariance)
n_labels = labels.max()

for i in range(n_labels + 1):
    print(‘Cluster %i: %s’ % ((i + 1), ', '.join(names[0][labels == i])))

Find a low-dimension embedding for visualization: find the best position of
the nodes (the stocks) on a 2D plane
We use a dense eigen_solver to achieve reproducibility (arpack is
initiated with random vectors that we don’t control). In addition, we
use a large number of neighbors to capture the large-scale structure.
node_position_model = manifold.LocallyLinearEmbedding(
n_components=2, eigen_solver=‘dense’, n_neighbors=6)

embedding = node_position_model.fit_transform(X.T).T

Visualization
plt.figure(1, facecolor=‘w’, figsize=(10, 8))
plt.clf()
ax = plt.axes([0., 0., 1., 1.])
plt.axis(‘off’)

Display a graph of the partial correlations
partial_correlations = edge_model.precision_.copy()
d = 1 / np.sqrt(np.diag(partial_correlations))
partial_correlations *= d
partial_correlations *= d[:, np.newaxis]
non_zero = (np.abs(np.triu(partial_correlations, k=1)) > 0.02) #np.triu返回上角矩阵

Plot the nodes using the coordinates of our embedding
#x=embedding[0],y=embedding[1],画出代表股票的点
plt.scatter(embedding[0], embedding[1], s=100 * d ** 2, c=labels,
cmap=plt.cm.nipy_spectral)

Plot the edges
start_idx, end_idx = np.where(non_zero)#以元组形式返回值为true的坐标

a sequence of (line0, line1, line2), where::
linen = (x0, y0), (x1, y1), … (xm, ym)
segments = [[embedding[:, start], embedding[:, stop]]
for start, stop in zip(start_idx, end_idx)]
values = np.abs(partial_correlations[non_zero]) #行列式子变换后的矩阵的非零值
lc = LineCollection(segments,
zorder=0, cmap=plt.cm.hot_r,
norm=plt.Normalize(0, .7 * values.max()))
lc.set_array(values)
lc.set_linewidths(10 * values)
ax.add_collection(lc)

Add a label to each node. The challenge here is that we want to
position the labels to avoid overlap with other labels
#x,y为表示股票坐标的点的坐标
for index, (name, label, (x, y)) in enumerate(
zip(names[0], labels, embedding.T)):
